<a href="https://colab.research.google.com/github/harim061/KoBERT/blob/main/kobert_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292699 sha256=1151dfc137606e4727f9375b42c2c15f2c844a800f0755967f642056d8fc1aec
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-p47un6hk/kobert-tokenizer_d398782a21634761af071c13936f51e3
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-p47un6hk/kobert-tokenizer_d398782a21634761af071c13936f51e3
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=e00f427414db9b9ce229ab5572900f6df94ff35140fc7630f12d8a30ae385b85
  Stored in directory: /tmp/pip-ephem-wheel-cache-p7_n6hmj/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [4]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# GPU 사용 시
device = torch.device("cuda:0")

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
import pandas as pd
data = pd.read_csv('5data.csv',encoding="cp949")

In [9]:
data.loc[(data['상황'] == "happiness"), '상황'] = 0 # happiness
data.loc[(data['상황'] == "neutral"), '상황'] = 1 # neutral
data.loc[(data['상황'] == "sadness"), '상황'] = 2 # sadness
data.loc[(data['상황'] == "angry"), '상황'] = 3 # angry
data.loc[(data['상황'] == "surprise"), '상황'] = 4 # surprise
data.loc[(data['상황'] == "disgust"), '상황'] = 5 # disgust
data.loc[(data['상황'] == "fear"), '상황'] = 6 # fear



In [11]:
data_list = []
for ques, label in zip(data['발화문'], data['상황'])  :
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [12]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')


In [13]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [14]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [21]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [22]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tokenizer, vocab,  max_len, True, False)

In [23]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [25]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# train

In [26]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-26-0cd8b201d8af>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0698535442352295 train acc 0.109375
epoch 1 batch id 201 loss 0.1579722762107849 train acc 0.8048818407960199
epoch 1 train acc 0.8272890946502057


<ipython-input-26-0cd8b201d8af>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/61 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9179961943793911


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.023769427090883255 train acc 1.0
epoch 5 batch id 201 loss 0.006661931052803993 train acc 0.9838308457711443
epoch 5 train acc 0.9845679012345679


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 test acc 0.9174399882903981


In [27]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 1:
                test_eval.append("중립")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 3:
                test_eval.append("화남")
            elif np.argmax(logits) == 4:
                test_eval.append("놀람")
            elif np.argmax(logits) == 5:
                test_eval.append("역겨움")
            elif np.argmax(logits) == 6:
                test_eval.append("무서움")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [28]:
#질문 무한반복하기! 0 입력시 종료
# end = 1
# while end == 1 :
#     sentence = input("하고싶은 말을 입력해주세요 : ")
#     if sentence == "0" :
#         break
#     predict(sentence)
#     print("\n")

# 자동화시키기

In [29]:
data_result = pd.read_csv('댓글.csv',encoding='cp949')


In [30]:
dataR = data_result.loc[:,'댓글']


In [31]:
for i in dataR:
    print(i+" : ",end='')
    predict(i)
    print('\n')

몬베베되고 첫인상이랑 젤 다른 멤버, 툭툭나오는 애교들이 진짜 너무 귀여움. 우리 고앵이 : 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용에서 기쁨 느껴집니다.


04:50 창균이 웃는 거 개귀엽다 : >> 입력하신 내용에서 놀람 느껴집니다.


몽베베 눈에는 그저 깜찍 겸둥 큐티 아기 막내인데 : >> 입력하신 내용에서 기쁨 느껴집니다.


몬엑 생기기전에 리얼리티 노머시부터 봤던 사람인데 진심 창균님 정말 진중하고 착하고 대단한 사람임,, 랩도 잘하고 목소리도 좋고 잘생기고 랩도 잘함 ㅠㅠ : >> 입력하신 내용에서 슬픔 느껴집니다.


처음보는 사람들에게는 막내같지 않겠지만 형들과 몬베베에게는 엄청난 사랑과 우쭈쭈를 받는 활동명을 잃어버린 균이 : >> 입력하신 내용에서 슬픔 느껴집니다.


1:07 이와중에 허니 진짜 꾸꾸까까 몇 천 번째 보는데 최강 귀여움.....뭐냐 이주헌 진짜 겁나 사랑한다???? : >> 입력하신 내용에서 기쁨 느껴집니다.


아 몬엑분들 왤케 귀여우세요...?; 애교가 아주 몸에 베여있네ㅠ 아 진짜 사람 보쌈해가고싶게? : >> 입력하신 내용에서 중립 느껴집니다.


이게 머선일이고 우리 창균이 분량 갱장히 조으다ㅠㅠㅠㅠ : >> 입력하신 내용에서 역겨움 느껴집니다.


Im Changkyun: Producer, lyricist, rapper, vocalist, dancer, model and forever our beloved maknae. Please release your new album soon!! : >> 입력하신 내용에서 화남 느껴집니다.


9:01 너무 귀여워서 심장이 멎을뻔…ㅎ하ㅏㅠ : >> 입력하신 내용에서 놀람 느껴집니다.


독보적인 시크남 아엠 ?? 독보적 케릭터 귀욤과 터프함도 겸비한 최애 연예인  주허니 : >> 입력하신 내용에서 기쁨 느껴집니다.


Hahahah that legendary clip of jooheon singing and changkyun just being shocked of his energy  : >> 입력하신 내용에서 슬픔 느껴집니다.


우리 짱규니 말예쁘고 다정한디ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ다

In [33]:
data_result2= pd.read_csv('result.csv')

In [34]:
dataR2 = data_result2.loc[:,'COMMENT']

In [35]:
for i in dataR2:
    print(i+" : ",end='')
    predict(i)
    print('\n')

유사범죄를 막을수 있는 가장 간단한 방법은 살인마에게 사형선고를 하고 사형집행을 하는것 뿐입니다.
인류애 넘치시는 대한민국 판사님들은 부산돌려차기남에게도 불우한 가정환경을 감형사유로 들어 감형을 해주었습니다.
살인마를 괴물로 키우는것은 어쩌면 대한민국 판사님들 덕분입니다. : 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용에서 화남 느껴집니다.


판사님들의 큰 공이죠
정상적인 사람도 일부가 살기위해 가해자가 되길 선택하게 만드는 유도성까지 퍼펙트 : >> 입력하신 내용에서 화남 느껴집니다.


법적처벌을 개의치않았다잖아요. 영상에나오는데..잡혀서 사형당한다한들 신경쓰지않는 유형이니 전혀다른 해결책이 필요한부분이죠 : >> 입력하신 내용에서 화남 느껴집니다.


ㅈ ㅏ ㅍ ㅖ 있냐?  방송 안봤어? 전문가가 그거 가지고 막을수 없다잖아 : >> 입력하신 내용에서 화남 느껴집니다.


​ @user-bm2ds5go2f 그거라도 하면 많이 줄겠지 : >> 입력하신 내용에서 슬픔 느껴집니다.


 @user-bm2ds5go2f 안되겠지하고 안바뀌면 어떻게 하실거죠 가만히 있다 찔려 사망하실건가요 아무것도 하지않으면 바뀌지 않습니다 : >> 입력하신 내용에서 화남 느껴집니다.


 @user-ic8kw9qw2s  
그러니까 허구헌날 일 벌어지고 
사형 지껄여바야
저런 범죄 예방엔 
아무런 도움이 안된다고
아니 방송 못 들었어?
사패나 소패 인적 연구가 필요하다고~ : >> 입력하신 내용에서 중립 느껴집니다.


뒤가없는인간인데 사형이고 지랄이고 신경쓰겠냐고ㅋㅋㅋ : >> 입력하신 내용에서 화남 느껴집니다.


말 그대로 어린아이도 생각해 낼 수 있는 가장 간단한 방법

근본 원인제거에는 아무짝에도 쓸모없는 생각

범죄의 근원이 무엇인지 연구하고 해결하는데에 투자를 해야지

범죄자만 나무라면 범죄자는 어디선가 또 나오는 거임

사회가 건강한데 그 속에서 범죄자가 나온 게 아니고

사회가 건강하지 않으니 그 속에서 범죄자가 나오는 거임

범죄자 처벌은 처벌이고 문제 해결하려면 사회문제를 고쳐야 함 : >> 입력하신 내용에서 화남 느껴집니다.


​ @user-bm2ds5go2f 최소한 재범은 못하지 않겠음? : >> 입력하신 내용에서 중립 느껴집니다.


사실상 전과 17범인데 완전 당당하게 거리를 활보할 수가 있는 미친 세상이다 : >> 입력하신 내용에서 중립 느껴집니

KeyboardInterrupt: ignored